In [3]:
import numpy as np
import pandas as pd
from functools import reduce
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
import plotly.graph_objects as go
import math
from scipy.stats import zscore

In [6]:
#loading data

asset_pchange_list = []
asset_price_list = []
asset_name_list = []

df = pd.read_excel('Data.xlsx', sheet_name=0)
date_list = np.flip(df.iloc[:,0].to_numpy().astype('datetime64[D]'))
num_of_asset = len(df.columns)
for i in range(1, num_of_asset):
    asset_array = df.iloc[:,i].to_numpy()
    mask = ~np.isnan(asset_array)
    asset_array_no_na = asset_array[mask]
    asset_price_list.append(np.flip(asset_array_no_na))
    asset_name_list.append(df.columns[i])

p_date_list = date_list[1:]
for i in range(len(asset_price_list)):
    asset_price = asset_price_list[i]
    print(asset_name_list[i], i)
    if i < 12 or (len(asset_price_list)>20 and i<19):
    
        try:
            diff_list = np.diff(asset_price) / asset_price[:-1]
        except:
            print(i)
    else:
        diff_list = np.diff(asset_price)
    asset_pchange_list.append(diff_list)



SPX index 0
MXEF Index 1
USCRWTIC Index 2
XAU Curncy 3
LMCADS03 Comdty 4
WEATTKHR index 5
USDEUR Curncy 6
USDAUD Curncy 7
USDKRW Curncy 8
USDINR Curncy 9
MOVE Index 10
VIX Index 11
USGG2YR Index 12
USGG10YR Index 13
USGG5YR INDEX 14
USYC2Y10 Index 15
USYC5Y30 Index 16


In [7]:
class Fix_TSMD():
    """Time series motif discovey (TSMD) class. To find out the period in the history that is the most similar to the one you input"""
    def __init__(self, price_lists, price_date_list, data_lists, data_date_list, asset_name_list, target_index, windows=10, n_closest=5, is_p_change = True):
        self.__price_lists = price_lists
        self.__price_date_list = price_date_list
        self.__data_lists = data_lists
        self.__data_date_list = data_date_list
        self.__target_index = target_index
        self.__windows = windows
        self.__n_closest = n_closest
        self.__is_p_change = is_p_change
        self.__asset_name_list = asset_name_list

        before_target_data_lists = np.array(data_lists)[:, :target_index]
        target_data_lists = np.array(data_lists)[:, target_index:target_index+windows]
        self.__before_target_data_lists = before_target_data_lists
        self.__target_data_lists = target_data_lists

    def fit(self, sd = False):
        """Fitting the model and discover the period. With root of sum of square value to quantify the similarity between diffferent period"""
        all_asset_e_dist_list = []
        windows = self.__windows
        is_p_change = self.__is_p_change
        before_target_data_lists = self.__before_target_data_lists
        n_closest = self.__n_closest
        target_data_lists = self.__target_data_lists
        for i in range(len(before_target_data_lists)):
            e_dist_list = []
            asset_data = before_target_data_lists[i]
            target_data = target_data_lists[i]
            for j in range(len(asset_data) - windows + 1):
                if is_p_change:
                    compare_data = asset_data[j:j+windows]
                    if sd:
                        compare_data = zscore(compare_data)
                        target_data = zscore(target_data)
                    e_dist = np.square(np.array(compare_data) - np.array(target_data))
                    n = len(e_dist)
                    indices = np.arange(n)
                    factors = indices + 1
                    weighted_e_dist_arr = e_dist * factors
                    weighted_e_dist = np.sqrt(np.sum(weighted_e_dist_arr))


                    e_dist = np.linalg.norm(np.array(compare_data) - np.array(target_data))
                    e_dist_list.append(weighted_e_dist)
                else:
                    compare_data = TimeSeriesScalerMeanVariance().fit_transform([asset_data[j:j+windows]])[0]
                    e_dist = np.linalg.norm(np.array(compare_data) - TimeSeriesScalerMeanVariance().fit_transform([target_data])[0])
                    e_dist_list.append(e_dist)
            all_asset_e_dist_list.append(e_dist_list)
        cumulative_asset_e_dist_list = reduce(np.add, all_asset_e_dist_list)
        n_smallest_e_dist_list = np.partition(cumulative_asset_e_dist_list, n_closest)[:n_closest]
        n_smallest_index = np.where(np.isin(cumulative_asset_e_dist_list, n_smallest_e_dist_list))[0]
        self.cumulative_asset_e_dist_list = cumulative_asset_e_dist_list
        self.n_smallest_e_dist_list = n_smallest_e_dist_list
        self.n_smallest_index = n_smallest_index
    def plot(self, show_mean = True, show_all = True):
        """Plotting the period, prediction and original data"""
        is_p_change = self.__is_p_change
        asset_price_lists = self.__price_lists
        asset_price_date_list = self.__price_date_list

        target_index = self.__target_index
        windows = self.__windows
        n_smallest_index = self.n_smallest_index
        asset_name_list = self.__asset_name_list
        all_prediction_list = self.__all_prediction_list
        new_date_list = self.__new_date_list
        extra_windows = 0
        
        if is_p_change:
            extra_windows = 1
        for i in range(len(asset_price_lists)):
            
            asset_price_list = asset_price_lists[i]
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=asset_price_date_list[:len(asset_price_list)], y=asset_price_list, name='index',  mode='lines'))
            for j in range(len(n_smallest_index)):
                index = n_smallest_index[j]
                fig.add_trace(go.Scatter(x=asset_price_date_list[index:index+windows+extra_windows], y=asset_price_list[index:index+windows+extra_windows], name= f"similar {j}", line = dict(color='red'),  mode='lines'))
                if show_all:
                    fig.add_trace(go.Scatter(x=new_date_list, y=all_prediction_list[i][j], name= f"predict {j}", line = dict(color='gray'),  mode='lines'))
            
            mean_list = self.__all_predict_mean_list[i]
            if show_mean:
                fig.add_trace(go.Scatter(x=new_date_list, y=mean_list, name= f"mean predict", line = dict(color='green'),  mode='lines'))
            fig.add_trace(go.Scatter(x=asset_price_date_list[target_index:target_index+windows+extra_windows], y=asset_price_list[target_index:target_index+windows+extra_windows], name= f"search target", line = dict(color='yellow'),  mode='lines'))
            
            fig.update_layout(title=asset_name_list[i],
                   xaxis_title='Date',
                   yaxis_title='Value')
            fig.show()

        pass

    def transform(self, n_data_points):
        """Made prediction based on the period following the most similar one"""
        self.__n_data_points = n_data_points 
        n_smallest_index = self.n_smallest_index
        data_lists = self.__data_lists
        data_date_list = self.__data_date_list
        
        price_lists = self.__price_lists
        asset_price_date_list = self.__price_date_list

        is_p_change = self.__is_p_change
        windows = self.__windows 
        target_index = self.__target_index
              
        all_prediction_list = []
        all_predict_mean_list = []
        all_predict_percentage_change_list =[]

        for i in range(len(data_lists)):
            data_list = data_lists[i]
            price_list = price_lists[i]
            asset_prediction_list = []
            predict_percentage_change_list = []
            for index in n_smallest_index:
                latest_price = price_list[target_index + windows]
                latest_date = asset_price_date_list[target_index + windows]
                new_data_list = [latest_price]
                new_date_list = [latest_date]
                for j in range(n_data_points - 1):
                    if i < 12 or (len(asset_price_list)>20 and i<19):
                    
                        latest_price *= (1 + data_list[index+windows+j])
                    else:
                        latest_price += data_list[index+windows+j]

                    latest_date += np.timedelta64(7, 'D')
                    new_date_list.append(latest_date)
                    new_data_list.append(latest_price)
                asset_prediction_list.append(new_data_list)
                predict_percentage_change_list.append(data_list[index+windows:index+windows+n_data_points])
            
            np_all_prediction_list = np.array(asset_prediction_list)
            mean_list = np.mean(np_all_prediction_list, axis=0)
            all_predict_mean_list.append(mean_list)
            all_prediction_list.append(asset_prediction_list)
            predict_percentage_change_mean = np.mean(np.array(predict_percentage_change_list), axis=0)
            all_predict_percentage_change_list.append(predict_percentage_change_mean)
        new_date_list = np.array(new_date_list).astype('datetime64[D]')
        self.__all_prediction_list = all_prediction_list
        self.__new_date_list = new_date_list
        self.__all_predict_mean_list = all_predict_mean_list
        self.__all_predict_percentage_change_list = np.array(all_predict_percentage_change_list)
    def plot_e_dist(self):
        cumulative_asset_e_dist_list = self.cumulative_asset_e_dist_list
        price_date_list = self.__price_date_list

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=price_date_list[:len(cumulative_asset_e_dist_list)], y=cumulative_asset_e_dist_list, name='similarity'))
        fig.update_layout(title='Similarity',
                   xaxis_title='Date',
                   yaxis_title='1/similarity')
        fig.show()

    def score(self):

        """Rate the prediction result using root of sum of square"""
        all_predict_percentage_change_list = self.__all_predict_percentage_change_list
        windows = self.__windows
        target_index = self.__target_index
        data_lists = self.__data_lists
        n_data_points = self.__n_data_points
        total_sse = 0
        for i in range(len(data_lists)):
            mean = all_predict_percentage_change_list[i]
            data_list = data_lists[i][target_index+windows:target_index+windows+len(mean)]
            e_dist = np.square(np.array(mean) - np.array(data_list))

            n = len(e_dist)
            indices = np.arange(n)
            factors = 1
            weighted_e_dist_arr = e_dist * factors
            weighted_e_dist = np.sqrt(np.sum(weighted_e_dist_arr))
            total_sse += weighted_e_dist
        self.total_sse = total_sse
        self.sse_per_n_forecast = total_sse/n_data_points
    

In [8]:
fix_tsmd = Fix_TSMD(asset_price_list, date_list, asset_pchange_list, p_date_list, asset_name_list, 1216, 14, 5, True)
fix_tsmd.fit()
fix_tsmd.transform(16)
fix_tsmd.plot(True, True)

In [9]:
class Flex_TSMD():
    """Another version of time series motif discovery model. But with a flexible searching windows size. In order to capture the similar pattern with time difference"""
    def __init__(self, asset_price_lists, asset_price_date_list, asset_percentage_change_lists, asset_percentage_change_date_list, asset_name_list):
        self.__asset_price_lists = asset_price_lists
        self.__asset_price_date_list = asset_price_date_list
        self.__asset_percentage_change_lists = asset_percentage_change_lists
        self.__asset_percentage_change_date_list = asset_percentage_change_date_list
        self.__asset_name_list = asset_name_list
    
    def fit(self, target_index, target_windows, n_similar, search_windows_range):
        self.__target_index = target_index
        self.__target_windows = target_windows
        asset_percentage_change_lists = self.__asset_percentage_change_lists
        before_target_data_lists = np.array(asset_percentage_change_lists)[:, :target_index]
        target_data_lists = np.array(asset_percentage_change_lists)[:, target_index:target_index+target_windows]
        self.__before_target_data_lists = before_target_data_lists
        
        self.__target_data_lists = target_data_lists

        num_of_asset = len(before_target_data_lists)
        num_of_data = len(before_target_data_lists[0])
        search_windows_list = search_windows_range

        all_sum_of_error_list = np.array([])
        all_details_list = []

        for search_windows in search_windows_list:
            for i in range(num_of_data - search_windows + 1):

                sum_of_error = 0
                for j in range(num_of_asset):
                    target_data = target_data_lists[j]
                    search_data = before_target_data_lists[j][i:i+search_windows]
                    resized_search_data = np.ravel(TimeSeriesResampler(sz=target_windows).fit_transform(search_data)[0])
                    e_dist = np.square(np.array(target_data) - np.array(resized_search_data))

                    n = len(e_dist)
                    indices = np.arange(n)
                    # factors = indices + 1
                    factors = 1
                    weighted_e_dist_arr = e_dist * factors
                    weighted_e_dist = np.sqrt(np.sum(weighted_e_dist_arr))
                    sum_of_error += weighted_e_dist
                all_sum_of_error_list = np.append(all_sum_of_error_list, sum_of_error)
                all_details_list.append({"search_windows": search_windows, "index": i, "sum_of_error": sum_of_error})
        n_smallest_e_dist_list = np.partition(all_sum_of_error_list, n_similar)[:n_similar]
        n_smallest_index = np.where(np.isin(all_sum_of_error_list, n_smallest_e_dist_list))[0]
        self.__details_list = np.take(all_details_list, n_smallest_index)
        
    def transform(self, n_data_points):
        details_list = self.__details_list
        asset_price_lists = self.__asset_price_lists
        asset_percentage_change_lists = self.__asset_percentage_change_lists
        asset_price_date_list = self.__asset_price_date_list
        target_index = self.__target_index
        target_windows = self.__target_windows
        self.__n_data_points = n_data_points

        all_prediction_list = []
        all_predict_mean_list = []
        all_predict_percentage_change_list = []

        num_of_asset = len(asset_percentage_change_lists)
        for i in range(num_of_asset):
            percentage_change_list = asset_percentage_change_lists[i]
            price_list = asset_price_lists[i]

            asset_prediction_lists = []
            predict_percentage_change_list = []
            for details in details_list:
                latest_price = price_list[target_index + target_windows]
                latest_date = asset_price_date_list[target_index + target_windows]
                new_data_list = np.array([latest_price])
                new_date_list = np.array([latest_date])
                start_index = details["index"]
                search_windows = details["search_windows"]
                scale = search_windows/target_windows
                search_percentage_change_list = percentage_change_list[start_index+search_windows:start_index+search_windows+math.ceil(n_data_points*scale)]
                resized_percentage_change_list = np.ravel(TimeSeriesResampler(sz=n_data_points).fit_transform(search_percentage_change_list)[0])
                for resized_percentage_change in resized_percentage_change_list[:-1]:
                    if i < 12 or (len(asset_price_list)>20 and i<19):
                    
                        latest_price *= (1 + resized_percentage_change)
                    else:
                        latest_price += resized_percentage_change

                    latest_date += np.timedelta64(7, 'D')
                    new_date_list = np.append(new_date_list, latest_date)
                    new_data_list = np.append(new_data_list, latest_price)
                
                asset_prediction_lists.append(new_data_list)
                predict_percentage_change_list.append(resized_percentage_change_list)
            np_all_prediction_list = np.array(asset_prediction_lists)
            mean_list = np.mean(np_all_prediction_list, axis=0)
            all_predict_mean_list.append(mean_list)
            all_prediction_list.append(asset_prediction_lists)
            predict_percentage_change_mean = np.mean(np.array(predict_percentage_change_list), axis=0)
            all_predict_percentage_change_list.append(predict_percentage_change_mean)
        new_date_list = new_date_list.astype('datetime64[D]')
        self.__all_prediction_list = all_prediction_list
        self.__new_date_list = new_date_list
        self.__all_predict_mean_list = np.array(all_predict_mean_list)
        self.__all_predict_percentage_change_list = np.array(all_predict_percentage_change_list)
    def plot(self, show_mean = True, show_all = True):
        details_list = self.__details_list
        asset_price_lists = self.__asset_price_lists
        asset_price_date_list = self.__asset_price_date_list

        target_index = self.__target_index
        target_windows = self.__target_windows
        asset_name_list = self.__asset_name_list
        all_prediction_list = self.__all_prediction_list
        new_date_list = self.__new_date_list
        for i in range(len(asset_price_lists)):
            
            asset_price_list = asset_price_lists[i]
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=asset_price_date_list[:len(asset_price_list)], y=asset_price_list, name='index',  mode='lines'))
            for j in range(len(details_list)):
                details = details_list[j]
                index = details["index"]
                search_windows = details["search_windows"]
                fig.add_trace(go.Scatter(x=asset_price_date_list[index:index+search_windows+1], y=asset_price_list[index:index+search_windows+1], name= f"similar {j}", line = dict(color='red'),  mode='lines'))
                if show_all:
                    fig.add_trace(go.Scatter(x=new_date_list, y=all_prediction_list[i][j], name= f"predict {j}", line = dict(color='gray'),  mode='lines'))
            
            mean_list = self.__all_predict_mean_list[i]
            if show_mean:
                fig.add_trace(go.Scatter(x=new_date_list, y=mean_list, name= f"mean predict", line = dict(color='green'),  mode='lines'))
            fig.add_trace(go.Scatter(x=asset_price_date_list[target_index:target_index+target_windows+1], y=asset_price_list[target_index:target_index+target_windows+1], name= f"search target", line = dict(color='yellow'),  mode='lines'))
            
            fig.update_layout(title=asset_name_list[i],
                   xaxis_title='Date',
                   yaxis_title='Value')
            fig.show()
    def score(self):
        all_predict_percentage_change_list = self.__all_predict_percentage_change_list
        target_windows = self.__target_windows
        target_index = self.__target_index
        n_data_points = self.__n_data_points
        asset_percentage_change_lists = self.__asset_percentage_change_lists

        total_sse = 0
        for i in range(len(all_predict_percentage_change_list)):
            target_data = asset_percentage_change_lists[i][target_index+target_windows:target_index+target_windows+n_data_points]
            predict_percentage_change = all_predict_percentage_change_list[i]
            e_dist = np.square(np.array(target_data) - np.array(predict_percentage_change))

            n = len(e_dist)
            indices = np.arange(n)
            factors = 1
            weighted_e_dist_arr = e_dist * factors
            weighted_e_dist = np.sqrt(np.sum(weighted_e_dist_arr))
            total_sse += weighted_e_dist
        self.total_sse = total_sse
        self.sse_per_n_forecast = total_sse/n_data_points
        

In [10]:
flex_tsmd = Flex_TSMD(asset_price_list, date_list, asset_pchange_list, p_date_list, asset_name_list)
windows = 14
windows_range = [13, 14, 15, 16]
flex_tsmd.fit(1216, windows, 5, windows_range)
flex_tsmd.transform(16)
flex_tsmd.plot()

In [7]:
#comparing the performance by aggregating the error with 600 samples.
#Result is flex: 37169.39113989008 flex: 37130.72140089765
indices = range(600, 1200)
fix_total_rss = 0
fix_rss_list = []

flex_total_rss = 0
flex_rss_list = []
for index in indices:
    print(index)
    fix_tsmd = Fix_TSMD(asset_price_list, date_list, asset_pchange_list, p_date_list, asset_name_list, index, 14, 5, True)
    fix_tsmd.fit()
    fix_tsmd.transform(16)
    fix_tsmd.score()
    fix_total_rss += fix_tsmd.total_sse
    fix_rss_list.append(fix_tsmd.total_sse)
    print("Fix RSS: ", fix_tsmd.total_sse)

    flex_tsmd = Flex_TSMD(asset_price_list, date_list, asset_pchange_list, p_date_list, asset_name_list)
    windows = 14
    windows_range = [13, 14, 15, 16]
    flex_tsmd.fit(index, windows, 5, windows_range)
    flex_tsmd.transform(16)
    flex_tsmd.score()
    flex_total_rss += flex_tsmd.total_sse
    flex_rss_list.append(flex_tsmd.total_sse)
    print("Flex RSS: ", flex_tsmd.total_sse)

600
Fix RSS:  87.12090661741155
Flex RSS:  86.39064973085922
601
Fix RSS:  87.98100659105314
Flex RSS:  90.51608473389149
602
Fix RSS:  85.09798589969897
Flex RSS:  87.18875299506317
603
Fix RSS:  85.83066716427027
Flex RSS:  83.39755592448192
604
Fix RSS:  79.69019142025277
Flex RSS:  91.92385216506672
605
Fix RSS:  75.80029781481517
Flex RSS:  82.37724148611403
606
Fix RSS:  76.86528815725359
Flex RSS:  81.90779615159781
607
Fix RSS:  76.51138674468204
Flex RSS:  95.98861103582163
608
Fix RSS:  79.07957497777943
Flex RSS:  91.81258733522986
609
Fix RSS:  69.747152527776
Flex RSS:  86.15454918219584
610
Fix RSS:  73.06248898869745
Flex RSS:  77.76564963623822
611
Fix RSS:  67.69359636346941
Flex RSS:  66.53295572448336
612
Fix RSS:  65.03334240768339
Flex RSS:  69.63552635912065
613
Fix RSS:  66.52846195263638
Flex RSS:  67.76558564470835
614
Fix RSS:  62.86266554188897
Flex RSS:  64.78695021789123
615
Fix RSS:  67.34608100451445
Flex RSS:  70.61656670450188
616
Fix RSS:  67.650439633

In [8]:
print(fix_total_rss, flex_total_rss)

37169.39113989008 37130.72140089765


In [10]:
#Finding the best prediction and worst prediction
"""
Result:
Three smallest rss for fix:  [22.19654272 26.03595629 25.85674193] indices:  [376 379 380]
Three smallest rss for flex:  [26.55460189 25.96534284 26.29585247] indices:  [379 380 382]
Three largest rss for fix:  [120.04515272 126.31160578 121.44079058] indices:  [545 581 582]
Three largest rss for flex:  [122.478051   126.79342913 123.35180157] indices:  [555 584 592]
"""
np_fix_rss_list = np.array(fix_rss_list)
np_flex_rss_list = np.array(flex_rss_list)

three_smallest_fix_rss_list = np.partition(np_fix_rss_list, 3)[:3]
three_smallest_fix_index = np.where(np.isin(np_fix_rss_list, three_smallest_fix_rss_list))[0]

three_smallest_flex_rss_list = np.partition(np_flex_rss_list, 3)[:3]
three_smallest_flex_index = np.where(np.isin(np_flex_rss_list, three_smallest_flex_rss_list))[0]

print("Three smallest rss for fix: ", three_smallest_fix_rss_list, "indices: ", three_smallest_fix_index)
print("Three smallest rss for flex: ", three_smallest_flex_rss_list, "indices: ", three_smallest_flex_index)

three_largest_fix_rss_list = np.partition(np_fix_rss_list, -3)[-3:]
three_largest_fix_index = np.where(np.isin(np_fix_rss_list, three_largest_fix_rss_list))[0]

three_largest_flex_rss_list = np.partition(np_flex_rss_list, -3)[-3:]
three_largest_flex_index = np.where(np.isin(np_flex_rss_list, three_largest_flex_rss_list))[0]

print("Three largest rss for fix: ", three_largest_fix_rss_list, "indices: ", three_largest_fix_index)
print("Three largest rss for flex: ", three_largest_flex_rss_list, "indices: ", three_largest_flex_index)

Three smallest rss for fix:  [22.19654272 26.03595629 25.85674193] indices:  [376 379 380]
Three smallest rss for flex:  [26.55460189 25.96534284 26.29585247] indices:  [379 380 382]
Three largest rss for fix:  [120.04515272 126.31160578 121.44079058] indices:  [545 581 582]
Three largest rss for flex:  [122.478051   126.79342913 123.35180157] indices:  [555 584 592]


In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=p_date_list[600:1200], y=fix_rss_list, name='Fix rss time graph',  mode='lines'))
fig.add_trace(go.Scatter(x=p_date_list[600:1200], y=flex_rss_list, name='Flex rss time graph',  mode='lines'))
fig.update_layout(title="Fix and flex rss time graph",
                   xaxis_title='Date',
                   yaxis_title='Rss')
fig.show()